### Tokenizacja

Poważnym problemem jest w jaki sposób tworzyć tokeny z tekstu:
* znaki oddzielone spacjami?
* znaki oddzielone na znakach interpunkcyjnych?
* rozdzielanie na podstawie regexów?
* inne metody?

#### Oddzielanie spacjami
jest to najprostszy sposób, ale też nie najlepszy, ponieważ np. słowo "won't" będzie jednym tokenem a samo "n't" można też dodać jako oddzielny token

#### oddzielanie na znakach interpunkcyjnych i spacjach
Tutaj dodatkowo rozdzielamy spacją wszystkie znaki interpunkcyjne czyli np. "Ala ma psa." -> \["Ala", "ma", "psa", "."\]

#### oddzielanie na regexach
jest to połączenie poprzednich metod i dodanie specjalnych fraz np. oddzielanie "n't" w języku angielskim.

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize


In [ ]:
word_tokenize("I didn't want to come.")


In [ ]:
text = (
    "Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium,"
    " totam rem aperiam, eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo. "
    "Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos qui "
    "ratione voluptatem sequi nesciunt. Neque porro quisquam est, qui dolorem ipsum quia dolor sit amet, consectetur, adipisci velit, "
    "sed quia non numquam eius modi tempora incidunt ut labore et dolore magnam aliquam quaerat voluptatem. Ut enim ad minima veniam, quis "
    "nostrum exercitationem ullam corporis suscipit laboriosam, nisi ut aliquid ex ea commodi consequatur? Quis autem vel eum iure reprehenderit "
    "qui in ea voluptate velit esse quam nihil molestiae consequatur, vel illum qui dolorem eum fugiat quo voluptas nulla pariatur?"
)


In [ ]:
sent_tokenize(text)


#### Subwords tokenization
Wszystkie poprzednie metody mają problem z nowymi słowami, które mogą się pojawić podczas tokenizacji nowego tekstu. W przypadku poprzednich metod zastępuje się zazwyczaj słowa nie występujące w słowniku przez "\<unk\>". 

Kolejnym problemem jest wielkość słownika, im więcej słów chcemy posiadać tym większy musi być nasz słownik co prowadzi do coraz większych wymagań pamięciowych w celu operowania na tekstach. 

Problemy te są rozwiązywane przez tokenizatory, który dokonują podziału na tokeny, które nie są całymi słowami tylko ich fragmentami. W takich modelach z góry określa się wielkość słownika. Oczywiście powstaje pytanie jak wybierać ciągu znaków, które będą tokenami.

Warto na nie zwrócić uwagę bo wszystkie aktualnie najlepsze modele językowe oparte o sieci neuronowe z nich korzystają ;)

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenizer.tokenize("I have a new GPU!")


Z biblioteką [tokenizers](https://huggingface.co/docs/tokenizers/python/latest/) dodać wiele różnych rzeczy jak dodawanie specjalnych tokenów na początek koniec, wiele innych pretokenizatorów i wiele innych gotowych tokenizatorów.

#### Tokenizacja w Kerasie

In [ ]:
tokenization_layer = tf.keras.layers.TextVectorization(
    max_tokens=None,
    standardize="lower_and_strip_punctuation",
    split="whitespace",
    ngrams=None,
    output_mode="int",
    output_sequence_length=None,
    pad_to_max_tokens=False,
    vocabulary=None,
    idf_weights=None,
    sparse=False,
    ragged=False,
)


In [ ]:
text_dataset = ["foo", "bar", "baz"]
max_features = 5000
max_len = 4


In [ ]:
# Tworzymy warstwe tokenizującą
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=max_features, output_mode="int"
)
# Dopasujemy tokenizator do danych
vectorize_layer.adapt(text_dataset)


model = tf.keras.models.Sequential()


In [ ]:
vectorize_layer.get_vocabulary()


In [ ]:
model.add(vectorize_layer)


In [ ]:
# Jako pierwszą warstwę dajemy nasz tokenizator


# Teraz nasz model na wejściu będzie akceptował teksty
input_data = [["foo qux bar asf aljhg kljalk hbkla foo"], ["qux baz"]]
model.predict(input_data)


## Reprezentacja słów i sieci neuronowe czyli embeddingi
Embeddingi polegają na zmniejszeniu wymiaru danych tekstowych, aby zakodować ciąg słów o słowniku wielkości 50000 tworzymy macierz o wymiarach seq_len x 50000, co można się domyślić nie jest optymalne, embeddingi sprowadzają dane tekstowe do dużo niższego wymiaru np. 300.

Przykładami embeddingów są:
* Word2Vec
* GloVe
* FastText

### Word2Vec

Metody uczenia 

![Word2vec image](Grafika/word2vec_diagrams.png)

### Podobieństwo słów
Mając embeddingi słów można badać ich podobieństwo badając ich odległość w przestrzeni, w której się znajdują. Najczęściej wykorzystuje się do tego odległość cosinusową

\begin{align*}
    \text{cos\_sim}(A,B) = \frac{A\cdot B}{||A||||B||}=cos(\theta)
\end{align*}

która pokazuje jaki jest kąt między dwoma wektorami, jeżeli 0 wtedy mamy 1 i oznacza to że wektory są tak samo skierowane czyli są podobne.

In [ ]:
from gensim.models import Word2Vec

sentences = [
    ["this", "is", "the", "first", "sentence", "for", "word2vec"],
    ["this", "is", "the", "second", "sentence"],
    ["yet", "another", "sentence"],
    ["one", "more", "sentence"],
    ["and", "the", "final", "sentence"],
]
# size: (default 100) wymiar przestrzeni embeddingów.
# window: (default 5) okno które będzie wykorzystywane do predykcji lub będzie predykowane.
# min_count: (default 5) minimalna liczba wystąpień słowa aby było uwzględnione w słowniku.
# workers: (default 3) liczba wątków wykorzystana do uczenia.
# sg: (default 0 or CBOW) jaki algorytm ma być wykorzystany do uczenia 0-CBOW, 1-Skip-gram.
model = Word2Vec(sentences, min_count=1)


Jeśli $x_t$ to _one-got_ z jedynką na pozycji i to 
$$W\cdot w_t=W \cdot [0,\dots, 1_i,\dots, 0]^T=W^x[:,i]$$
Zatem przekształcenie to jest równoważne wzięciu i'tej kolumny macierzy wag.

In [ ]:
model.wv


In [ ]:
model.wv.key_to_index


In [ ]:
words = list(model.wv.key_to_index.keys())


In [ ]:
print(model.wv["sentence"])


In [ ]:
model.wv.key_to_index


In [ ]:
words


In [ ]:
X = model.wv[words]
X.shape


In [ ]:
X[0] == model.wv["sentence"]


In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt


In [ ]:
x_transformed = PCA(2).fit_transform(X)


In [ ]:
plt.scatter(x_transformed[:, 0], x_transformed[:, 1])

for i, word in enumerate(words):
    plt.annotate(word, xy=(x_transformed[i, 0], x_transformed[i, 1]))


##### wczytywanie gotowych embeddingów


In [ ]:
from gensim import downloader


In [ ]:
glove_vectors = downloader.load("glove-wiki-gigaword-100")


In [ ]:
result = glove_vectors.most_similar(
    positive=["woman", "king"], negative=["man"], topn=5
)
result


In [ ]:
glove_vectors["i", "have", "a"].shape


### embeddingi fasttext
W Word2Vec tworzymy embeddingi słów w celu stworzenia embeddingu więc nie jesteśmy w stanie otrzymać embeddingu słowa spoza naszego słownika. Fasttext inaczej pochodzi do tworzenia embeddingów, ponieważ słowa, które są do siebie podobne(co do ogległości edycyjnej) powinny mieć podobne embeddingi postanowiono tworzyć je na podstawie n-gramów na znakach (dla n od 3 do 6). Jak to działa?
1) Dodajemy na początek słowa '<' a na koniec '>'.

![](Grafika/fasttext-angular-brackets.png)

2) tworzymy n-gramy dla słowa.
3) Ponieważ liczba n-gramów może być ogromna dlatego, zamiast trenować embeddingi dla każdego unikatowego n-grama, trenowane jest B (B-bucket size). Każdy n-gram jest przetwarzany przy użyciu funkcji hashującej do liczby całkowitej między 1 a B.
4) Do słownika dodajemy także słowa, które występują w zbiorze treningowym. Zatem mamy B+|V| embeddingów.

#### Jak trenowany jest fasttext?

Embeddingi są trenowane wykorzystując skip-gram z negatywnym próbkowaniem. Czyli na podstawie słowa chcemy przewidzieć słowa sąsiadujące. Ale embedding słowa na podstawie, którego chcemy przewidywać to suma n-gramów i embeddingu tego słowa.

![](Grafika/fasttext-negative-sampling-goal.png)



### Doc2Vec
Doc2Vec jest wykorzystaniem podobnego pomysłu co Word2Vec. Czyli na podstawie contekstu przewidujemy słowo. Ale skąd tutaj embedding dokumentu? Dodany jest dodatkowo embedding paragrafu jak na zdjęciu poniżej. Z dodatkiem tego embeddingu trenowany jest model  PV-DM(Distributed Memory version of Paragraph Vector) lub PV-DBOW(Words version of Paragraph Vector) (Podobny model do skip-gram).

![](Grafika/doc2vec_dbow.png) 
![](Grafika/doc2vec_skip_gram.png)

Jak zdobyć embedding nowego dokumentu? W tym celu zamrażane są wszystkie wagi sieci i jedyną zmienną jest embedding dokumentu, następnie ta zmienna jest aktualizowana trenując ją jak PV-DM lub PV-DBOW.

Możliwe też jest z wykorzystaniem Doc2Vec modelowanie gatunków. Zamiast unikatowego id można dodawać(dodatkowo lub jako jedyne wejście) tag związany z kategorią. W ten sposób otrzymamy embeddingi kategorii i po wyznaczeniu embeddingu danego dokumentu możemy powiedzieć do której kategorii on najprawdopodobniej należy.

![](Grafika/doc2vec_tag.png)

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# Tutaj możemy też podać kategorie
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)


[parametry klasy Doc2Vec](https://radimrehurek.com/gensim/models/doc2vec.html#introduction)

In [ ]:
common_texts


In [ ]:
x = model.infer_vector(["human", "interface", "computer"])


In [ ]:
model.dv.most_similar(x)


In [ ]:
model.mos


In [ ]:
model.similarity_unseen_docs(["human", "response"], ["computer", "response"])
